<a href="https://colab.research.google.com/github/npd2013/Mypublicmodels/blob/main/old_7_5_featureextraction_vectorspace_model_with_corefernce_resolve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SETUP1:-  
-Single document text summarization 

-frequency based method for scoring sentences 

-preprocessing applied are knowledge based normalization,  stopword removal and stemming.
-Using  CUE WORDS - actors in story.

-automatic evaluation using rouge and compression ratio analysis.

note:results stored in another file 


Algorithm steps :- 

1) Preprocessing of text- 

    1.1)Cleaning 

    1.1.1)Removal of unnecessary symbols/charachters like html tags ,extra spaces,extra newlines etc.
         
 
 1.2) Line segmentation
 
    -using indicnlp with own segmenter  
    -Divide text into lines and find token words using own word segmenter.
 1.3) word segmentation
    -Divide the lines into words.
    -words can be separated by whitespace or characters like quotes or comma.  use the punctuation list to exclude from the word.Only extract valid words 

  1.4) Stopword removal 
 
       -basic set of 15 words removed from lines. 
  1.5) Co-reference resolution
       - if current sentences contain referencewords
            - if previous two sentences contain 
               "actor" words or its variations
                - replace referenceword with actor word.

   
 1.5) Stemming
    -if word hyphenated (considered as joint word)
        -exit   

    -use static dictionary of standard words.
 
    -remove matching salutation from suffix word.
    -Check is it valid dictionary word or wrong spelled dictionary word 
       -add the word in lemmalist
       exit
    
    -if not valid dictionary word

       -look up prefix and trim if match found

       -look up replacement and trim if match found

       - look up longest suffix rules matching and trim if match found.
    
    -if trimmed word size>=3 
          - stemword = trimword
          - store the mapping of stemword and orginalword in lemmalist.
          - replace word with stemword found
  
1.6) Stem to lemma
   -for all words in lemmalist 
     - if 
                   
               
       

2)Make vector space representation -based on term frequency

    2.1) Compute tfidf using sklearn module of python.

    2.2) For each sentence - 

     2.2.1) Compute sentence score= total tfidf score of all words in sentence divide by number of words in sentence.
     2.2.2) If it is first or last sentence , increase its weightage ten times.
  
  2.3) For each sentence -

     3.2.1) Compute the average normalized score  : 
          Current sentence score divide by total sentence score of text.
      

 4) //Find top "n" scored sentences.

 4.1) Set n= number of lines in gold summary. 0r "x" percent of original text.    
 4.2) Summary_length=n
 4.3) Sorted_sentences=sort(sentences) based on their scores.
    
 5) //Generate the summary -

    for sentences = 1 to gold_summary_length 
      - if sentence in gold_summary  
           put sentence in system_summary. 
  
6)// Evaluate the summary-

    6.1) use the Rouge score 
      - Get the precision ,recall and F-measure for rouge-1 ,rouge2 and rouge-LCS.   

    6.2) Find comression ratio of text
       6.2.1) sentence compression ratio
        = total sentences in system_summary/total sentences in source text 
        6.2.2) Word compression ratio
        =total words in system_summary/total words in source 
  


In [1]:
#install needed packages 
#!pip install  gensim  #has word2vec model built, using english 

!pip install pyiwn
!pip install -U scikit-learn
!pip install rouge
!pip install indic-nlp-library


     |████████████████████████████████| 22.3MB 128kB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 40kB 4.1MB/s 
     |████████████████████████████████| 9.2MB 11.8MB/s 
     |████████████████████████████████| 552kB 36.4MB/s 
  Created wheel for sphinx-argparse: filename=sphinx_argparse-0.2.5-cp37-none-any.whl size=11552 sha256=521265843178d05fd72126d27a2054ea216fc65e9257db6573e435eb2a1d19c9
  Stored in directory: /root/.cache/pip/wheels/2a/18/1b/4990a1859da4edc77ab312bc2986c08d2733fb5713d06e44f5
Successfully built sphinx-argparse
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1


In [2]:
from google.colab import files #upload the files from local machine on the colab environment 
uploaded = files.upload()


Saving story1.txt to story1.txt
Saving story1-gold.txt to story1-gold.txt
Saving story1-topic.txt to story1-topic.txt
Saving story2.txt to story2.txt
Saving story2-gold.txt to story2-gold.txt
Saving story2-topic.txt to story2-topic.txt
Saving story3.txt to story3.txt
Saving story3-gold.txt to story3-gold.txt
Saving story3-topic.txt to story3-topic.txt
Saving story4.txt to story4.txt
Saving story4-gold.txt to story4-gold.txt
Saving story4-topic.txt to story4-topic.txt
Saving story5.txt to story5.txt
Saving story5-gold.txt to story5-gold.txt
Saving story5-topic.txt to story5-topic.txt
Saving story6.txt to story6.txt
Saving story6-gold.txt to story6-gold.txt
Saving story6-topic.txt to story6-topic.txt
Saving story7.txt to story7.txt
Saving story7-gold.txt to story7-gold.txt
Saving story7-topic.txt to story7-topic.txt
Saving story8.txt to story8.txt
Saving story8-gold.txt to story8-gold.txt
Saving story8-topic.txt to story8-topic.txt
Saving story9.txt to story9.txt
Saving story9-gold.txt t

In [3]:
!cp "/content/drive/My Drive/Colab Notebooks/guj_final_preporcess-v6.py" "guj_final_preporcess.py" #The preprocessing modules designed
!cp "/content/drive/My Drive/Colab Notebooks/Evaluation.py" "Evaluation.py"   # the evaluation module designed
!cp "/content/drive/My Drive/Colab Notebooks/guj_final_postprocessing-v1.py" "post.py"   # Get the post processing module 
!cp "/content/drive/My Drive/Colab Notebooks/my-dictionary-gujarati.csv" "my-dictionary-gujarati.csv" #the static dictionary of words in gujarati


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import io
import csv
from Evaluation import *
from post import *
import pandas as pd 
from guj_final_preporcess import * #to get the required preporcessing modules -from self made pipeline
import pyiwn

2021-06-25:11:56:14,3 INFO     [helpers.py:20] Downloading IndoWordNet data of size ~31 MB...


[██████████████████████████████████████████████████]

2021-06-25:11:56:15,529 INFO     [helpers.py:43] Extracting /root/iwn_data.tar.gz into /root...


2021-06-25:11:56:16,824 INFO     [helpers.py:48] Removing temporary zip file from /root/iwn_data.tar.gz
2021-06-25:11:56:16,835 INFO     [helpers.py:51] IndoWordNet data successfully downloaded at /root/iwn_data


In [304]:
#take all the needed inputs from user 
basename=input('Enter the basic name of file(without txt),other names wud be autogenerated  ')
filename=basename+'.txt'
topic_filename=basename+'-topic.txt'
#filename=input('Enter text file name to be summarized ') #Ex: file1.txt
analysefile_name=filename.replace('.txt','-analysis-Model1.txt')# to store the data for later analysis . Example- file1-results.txt 
features_filename=filename.replace('.txt','-Features-Model1.csv') # file-1-features.csv
filename_statistics_all='datastatisticsall--Model1 .csv'
#gold_filename=input("Enter name of file containing gold summary  ") #example file1-gold.txt ,assumes the name is having pattern ending with gold.txt 
summary_filename=filename.replace('gold.txt','-systemsummary--Model1 .txt') # example file1-gold-system-summary.txt


rougescorefile="Rougeresults--Model1-- .csv"
dictionaryfile="my-dictionary-gujarati.csv"

Enter the basic name of file(without txt),other names wud be autogenerated  story51


In [305]:
"""
print("enter modules you want to activate in current run ")
vectormodel=input("1- Using TFIDF technique  2- Using TF technique :: your choice")
stopwordflag=input("3-With stopword removal phase    4- Without stopword removal phase:: your choice")
stemmerflag=input("5- With stemmer phase     6- Without stemmer phase:: your choice ")
lemmaflag=input("7- With stem to lemma c3onversion.  8- Without stem to lemma conversion:: your choice")
cueflag=input("9- Using cuewords list 10-Without using cuewordslist" )
coreferenceflag=input("11- Using coreference resolution list 12-Without using coreference resolution")
"""
vectormodel='2'
stopwordflag='3'
stemmerflag='5'
lemmaflag='7'
cueflag='9'
coreferenceflag='11'



In [306]:
#read the input original file . 

guj_clean_html(filename,filename)# text cleaning remove the html tags and overwrite the original file 

guj_clean_html(topic_filename,topic_filename)

guj_clean_extra(filename,filename)#remove the extra symbols like multiple punctuations and overwrite original file

guj_clean_extra(topic_filename,topic_filename) 
#Display the given data in input files 
with io.open(filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file
analyzedfile_obj=open(analysefile_name,mode='w',encoding='UTF-8')
analyzedfile_obj.write("\n\n The original file is :------\n")
analyzedfile_obj.write(lines)




with io.open(topic_filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file

analyzedfile_obj.write("\n\n The topic source file given is :------\n")
analyzedfile_obj.write(lines)



# now find the statistics of originally given informations 
originalcorpora,totaloriginalsentences=guj_corpus_generate(filename) #make corpus of original data 


analyzedfile_obj.write("\n\n STATISTICS------>")
analyzedfile_obj.writelines("\n Original file sentences(RAW) - {0}\n".format(totaloriginalsentences))


###################################################################
########################  Step-1 ##################################
#####################  cleaning phase  ########################

#doing cleaning of file 
#remove newlines and other unnecessary data and clean it for feature extraction 
#to replace the data as per our format, replacing abbreviations dot with '*'
normfile="temp1.txt"
norm_gold_file="temp2.txt"
clean_topic_file=norm_topic_file="temp3.txt"

guj_clean_text(filename,normfile) 


guj_clean_text(topic_filename,clean_topic_file) #no further process needed for topic file, it is already cleaned.

#Now display the stats and make corpora after cleaning file and removing new lines . 
normcorpora,totalnormsentences=guj_corpus_generate(normfile)  

topiccorpora,totaltopicsentences=guj_corpus_generate(norm_topic_file)

analyzedfile_obj.write(" Segmented Lines in source -{0} \n ".format(totalnormsentences))

analyzedfile_obj.write(" the total sentences in normalized topic file are= {0} ".format(totaltopicsentences))
###################################################################
########################  Step-2 ##################################
#####################  Normalization and tokenization  for feature extraction  ########################

cleanfile="file.txt"
stemmedlist=[] #to store the stemmed words 
lemmalist=[] #to store the lemmas(dictionary) words found from vacabulory 
mydictionary=guj_makelist(dictionaryfile) #generate the dictionary of unique words from given file of gujarati words
vocab=guj_makevocab(normfile)

words_before_stopword_removal=len(vocab)

if (stopwordflag=='3'):
 cleanfile=guj_stopwordremoval(normfile,cleanfile) #remove stopwords done 

 



else :
  #print("stopwords are not removed")
  cleanfile=normfile # no stopword removal step done ,so clean file is directly the normalized one 



scorpora,stotalsent=guj_corpus_generate(cleanfile) #scorpora- the changed lines after stopword removal 


focus_list=guj_makelist(topic_filename)

## logic for co-reference resolution --
corpora,totalsent=guj_corpus_generate(cleanfile) #divide lines into sentences-
vocab_topic=guj_makevocab(clean_topic_file) #extra words to be included in the feature extraction ,based on topic

newlist=[]
actorlist=list(vocab_topic.keys())

stem_topic_filename="temp4.txt"
guj_stemmer_new(topic_filename,stem_topic_filename,mydictionary,actorlist)
stemtopic,totalstemline=guj_corpus_generate(stem_topic_filename)
extrawords=list(guj_makevocab(stem_topic_filename))
for word in extrawords:
 actorlist.append(word)
actorlist=set(actorlist) 



#Do co-reference resolution within previous 2 sentences
if (coreferenceflag=='11'):
  cleanfile=anaphoraresolution(corpora,actorlist,cleanfile)




vocab=guj_makevocab(cleanfile)
words_before_stemming=len(vocab)#after anaphora resolution vocab size

if (stemmerflag=='5'):
  cleanfile,stemmedlist,lemmalist=guj_stemmer_new(cleanfile,cleanfile,mydictionary,focus_list) #stem with help from dictionary ,and get back two lists 

stemcorpora,stemtotalsent=guj_corpus_generate(cleanfile) #stemcorpora- the changed lines after stemming 
vocab=guj_makevocab(cleanfile)
words_after_stemming=len(vocab)

if (lemmaflag=='7'):
  cleanfile=guj_stem_to_lemma(cleanfile,stemmedlist,lemmalist) #if have both stem and dictionary form of word ,convert to the dictionay form 

vocab=guj_makevocab(cleanfile)
words_after_lemma=len(vocab)











semgnet to make corpus for story51.txt
corpus now has sentences --> 
118
semgnet to make corpus for temp1.txt
corpus now has sentences --> 
118
semgnet to make corpus for temp3.txt
corpus now has sentences --> 
1
semgnet to make corpus for file.txt
corpus now has sentences --> 
118
semgnet to make corpus for file.txt
corpus now has sentences --> 
118
semgnet to make corpus for temp4.txt
corpus now has sentences --> 
1
semgnet to make corpus for file.txt
corpus now has sentences --> 
118
semgnet to make corpus for file.txt
corpus now has sentences --> 
118


In [307]:
#!!!!!!!!!!!!!!!!!!!For finding the actors / Focus group in story---- using both tf and tfidf to extract featuers  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#####

#Now working on the modified file to get various features 
#guj_clean_extra(finalfile,finalfile) #clean any residual marks in file 
#guj_clean_extra(preprocessedtopicfile,preprocessedtopicfile)


corpora,totalsent=guj_corpus_generate(cleanfile) #divide lines into sentences-
#print("\n Lines in source after all processing -{0} \n ".format(totalsent))
sent_pos_list=[] #to store the sentence position
sent_words_count=[]  # to store the total words in each sentence 
sent_number=0
for s in corpora:
  sent_pos_list.append(sent_number)
  sent_number+=1


  

# to generate vector represnetations  -----> TF 

vectorizer_tf = CountVectorizer(tokenizer = guj_mytokenizer,min_df=0.15,max_df=0.85) #frequency of words with atleast 15% frequency
 

try:
 cwm_tf = vectorizer_tf.fit_transform(corpora) #vectorize the corpus of input file
  
 n=0
 top_features_tf = vectorizer_tf.vocabulary_ #to store the features extracted
 
 for f in top_features_tf.keys():
   n+=1 #top features total count
 print("\ntop features and their values were{0} -TF".format(top_features_tf))
 pos=n
 tokens = vectorizer_tf.get_feature_names() #get the names of words used in vector   
except: #no features able to pass the criteria 
  pos=0
  top_features_tf={}


#### Extra logic for topic based vectorizor feature upadation ###########333
if (cueflag=='9'):

 for feature, index in vocab_topic.items(): #take all topic words from vocab and add in features 
    
    if feature not in top_features_tf.keys() or len(top_features_tf)==0:#add only if not already considered in top words or no words already in top
          top_features_tf[feature] = pos
          pos=pos+1
 # re-vectorize using both sets of features
 
 vectorizer_tf = CountVectorizer(vocabulary=top_features_tf,tokenizer = guj_mytokenizer)   
 cwm_tf = vectorizer_tf.fit_transform(corpora)
 tokens_tf = vectorizer_tf.get_feature_names() #get the names of words used in vector
 top_features_tf = vectorizer_tf.vocabulary_
 print("\n New top features after considering the actor words are {0} -TF".format(top_features_tf))
#### End of extra logic for adding separate topic based keywords in learning 

score_list_tf=[]
counts_tf=cwm_tf.sum(axis=1) #sums the sentence wise words' tfidf scores
scorelist_tf=list(counts_tf.flat)#get the list of sentencescores 


print("\n\n****The TF  based vector is formed ****")
print('\n The top features used in vector are -\n {0}-TF '.format(cwm_tf.shape))

#### similar steps for tfidf scores computation###
 ##############################################################
 #############################################################
# to generate vector represnetations  
vectorizer_tfidf = TfidfVectorizer(tokenizer = guj_mytokenizer,min_df=0.10,max_df=0.85) #frequency of words with atleast 15% frequency

try:
 cwm_tfidf = vectorizer_tfidf.fit_transform(corpora) #vectorize the corpus of input file
 
 n=0
 top_features_tfidf = vectorizer_tfidf.vocabulary_ #to store the features extracted
 
 for f in top_features_tfidf.keys():
   n+=1 #top features total count
 print("\ntop features and their values were{0} -".format(top_features_tfidf))
 pos=n
 tokens = vectorizer_tfidf.get_feature_names() #get the names of words used in vector   
except: #no features able to pass the criteria 
  pos=0
  top_features_tfidf={}
if (cueflag=='9'):

 for feature, index in vocab_topic.items(): #take all topic words from vocab and add in features 
    
    if feature not in top_features_tfidf.keys() or len(top_features_tfidf)==0:#add only if not already considered in top words or no words already in top
          top_features_tfidf[feature] = pos
          pos=pos+1
 # re-vectorize using both sets of features
 vectorizer_tfidf = TfidfVectorizer(vocabulary=top_features_tfidf,tokenizer = guj_mytokenizer)
  
 cwm_tfidf = vectorizer_tfidf.fit_transform(corpora)
 tokens_tfidf = vectorizer_tfidf.get_feature_names() #get the names of words used in vector
 top_features_tfidf = vectorizer_tfidf.vocabulary_
 print("\n New top features after considering the actor words are {0} in TFIDF-".format(top_features_tfidf))
#### End of extra logic for adding separate topic based keywords in learning 

score_list_tfidf=[]
counts_tfidf=cwm_tfidf.sum(axis=1) #sums the sentence wise words' tfidf scores
scorelist_tfidf=list(counts_tfidf.flat)#get the list of sentencescores 

print("\n\n****The TFIDF  based vector is formed ****")
print('\n The top features used in vector are -\n {0} -TFIDF'.format(cwm_tfidf.shape))

########################################################## 
################# ######Step3#############################
################ Compute sentence scores #################
##########################################################

totwords=[]

for s in corpora:
   count=0
   words=guj_mytokenizer(s)
   for c in words:
    count=count+1
   totwords.append(count)


list_of_tuples = list(zip(sent_pos_list,normcorpora,cwm_tf.toarray(),cwm_tfidf.toarray(),scorelist_tf,scorelist_tfidf,totwords))  
df=pd.DataFrame(list_of_tuples, columns=['sent_pos','source_lines','feature_TF','feature_TFIDF','totalsentcount_tf','totalsentcount_tfidf','totalwordsinsent'])

df['sent_score_tf']=df['totalsentcount_tf']/df['totalwordsinsent']
df['sent_score_tfidf']=df['totalsentcount_tfidf']/df['totalwordsinsent']
#based on sentence position , increase weightage of first and last and second last last sentences in txt 


df['bordersentences']='0' # Is current sentence start or end sentence in text? Default is no - 0 
df.at[0,'bordersentences']='1'
df.at[totalnormsentences-1,'bordersentences']='1'


df.to_csv(features_filename) 
df.to_csv(filename_statistics_all,mode='a',header=False) 


semgnet to make corpus for file.txt
corpus now has sentences --> 
118

top features and their values were{'સાહેબ': 1, 'નાથા': 0} -TF

 New top features after considering the actor words are {'સાહેબ': 1, 'નાથા': 0, 'મુખી': 2, 'બુધિયા': 3, 'સવિતા': 4} -TF


****The TF  based vector is formed ****

 The top features used in vector are -
 (118, 5)-TF 

top features and their values were{'સાહેબ': 4, 'ઘર': 1, 'આવતા': 0, 'સવિતા': 3, 'નાથા': 2} -

 New top features after considering the actor words are {'સાહેબ': 4, 'ઘર': 1, 'આવતા': 0, 'સવિતા': 3, 'નાથા': 2, 'મુખી': 5, 'બુધિયા': 6} in TFIDF-


****The TFIDF  based vector is formed ****

 The top features used in vector are -
 (118, 7) -TFIDF


In [308]:
analyzedfile_obj.close()



In [310]:
## code to download all data at end of execution .Uncomment it after all experiments done. 
'''
from google.colab import files

!zip -r /content/txtfiles.zip *.txt
files.download("txtfiles.zip")
!zip -r /content/csvfiles.zip *.csv
files.download("csvfiles.zip")
!zip -r /content/codesfiles.zip *.py
files.download("codesfiles.zip")
'''


  adding: file.txt (deflated 75%)
  adding: story10-analysis-Model1.txt (deflated 71%)
  adding: story10-gold.txt (deflated 71%)
  adding: story10-topic.txt (deflated 11%)
  adding: story10.txt (deflated 72%)
  adding: story11-analysis-Model1.txt (deflated 75%)
  adding: story11-gold.txt (deflated 75%)
  adding: story11-topic.txt (stored 0%)
  adding: story11.txt (deflated 76%)
  adding: story12-analysis-Model1.txt (deflated 76%)
  adding: story12-gold.txt (deflated 78%)
  adding: story12-topic.txt (deflated 2%)
  adding: story12.txt (deflated 78%)
  adding: story13-analysis-Model1.txt (deflated 75%)
  adding: story13-gold.txt (deflated 76%)
  adding: story13-topic.txt (stored 0%)
  adding: story13.txt (deflated 77%)
  adding: story14-analysis-Model1.txt (deflated 76%)
  adding: story14-gold.txt (deflated 77%)
  adding: story14-topic.txt (deflated 24%)
  adding: story14.txt (deflated 78%)
  adding: story15-analysis-Model1.txt (deflated 73%)
  adding: story15-gold.txt (deflated 73%)
  a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: datastatisticsall--Model1 .csv (deflated 79%)
  adding: lemmatized.csv (deflated 86%)
  adding: my-dictionary-gujarati.csv (deflated 79%)
  adding: stemmingdata.csv (deflated 78%)
  adding: story10-Features-Model1.csv (deflated 73%)
  adding: story11-Features-Model1.csv (deflated 75%)
  adding: story12-Features-Model1.csv (deflated 77%)
  adding: story13-Features-Model1.csv (deflated 77%)
  adding: story14-Features-Model1.csv (deflated 79%)
  adding: story15-Features-Model1.csv (deflated 76%)
  adding: story16-Features-Model1.csv (deflated 81%)
  adding: story17-Features-Model1.csv (deflated 77%)
  adding: story18-Features-Model1.csv (deflated 78%)
  adding: story19-Features-Model1.csv (deflated 78%)
  adding: story1-Features-Model1.csv (deflated 76%)
  adding: story20-Features-Model1.csv (deflated 77%)
  adding: story21-Features-Model1.csv (deflated 78%)
  adding: story22-Features-Model1.csv (deflated 77%)
  adding: story23-Features-Model1.csv (deflated 78%)
  adding: story2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: Evaluation.py (deflated 64%)
  adding: guj_final_preporcess.py (deflated 75%)
  adding: post.py (deflated 70%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>